In [1]:
%matplotlib inline
import os, sys, numpy as np, pandas as pd, tensorflow as tf, re, codecs, seaborn as sns, json, time, csv, datetime as dt
import pickle, collections, random, math, numbers, scipy.sparse as sp, matplotlib.pyplot as plt, scipy.sparse as sp
from pprint import pprint

def reload(mName):
    import importlib
    if mName in sys.modules:
        del sys.modules[mName]
    return importlib.import_module(mName)


from collections import deque, defaultdict, OrderedDict
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, minmax_scale
from matplotlib import pyplot as plt
plt.style.use('ggplot')

# classpath
ctx = os.path.abspath('..')
cps = [ctx]
_ = [sys.path.insert(0, cp) for cp in cps if cp not in sys.path]

# data path
datapath = '/'.join([ctx, 'data'])

seed = 88
utils = reload('utils.utils')
np.set_printoptions(precision=4, suppress=True, linewidth=100)
np.random.seed(seed)

h:\anaconda3\envs\ml_dl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# tf.decode_csv + tf.data.TextLineDataset

## Simple Data Preprocess

In [2]:
import datetime as dt

ratings = pd.read_csv("{}/ml-latest-small/ratings.csv".format(datapath))
ratings['timestamp'] = ratings.timestamp.map(dt.datetime.fromtimestamp).map(str)
ratings['ori_rating'] = ratings['rating']
ratings['rating'] = (ratings.rating >= 4).astype(int)
tr, te = utils.split_by_ratio(ratings)

movies = pd.read_csv("{}/ml-latest-small/movies.csv".format(datapath))
avg_rt = ratings.groupby("movieId", as_index=False).ori_rating.mean().rename(index=str, columns={'ori_rating': 'avg_rating'})
movies = movies.merge(avg_rt, how='left', on='movieId')
# movies.avg_rating.fillna(ratings.rating.mean())
movies["year"] = movies.title.str.findall("\(\s*(\d+)\s*\)").map(lambda lst: int(lst[-1]) if len(lst) else None)
# movies["year"] = minmax_scale(movies.year.fillna(movies.year.median()))

In [3]:
def preprocess(data, movie_trans, train_hist=None, is_train=True):
    queue = []
    data = data.merge(movie_trans, how="left", on="movieId")
    columns=["user_id", "query_movie_ids",
             "genres", "avg_rating", "year", "candidate_movie_id",
             "timestamp",
             "rating"]
    
    list2str = lambda lst: ','.join(map(str, lst))
    for u, df in data.groupby("userId"):
        df = df.sort_values("rating", ascending=False)
        if not is_train:
            user_movies_hist = train_hist.query("userId == {}".format(u)).movieId
        for i, (_, r) in enumerate(df.iterrows()):
            if is_train:
                query_hist = df.movieId[:i].tolist() + df.movieId[i + 1:].tolist()
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
            else:
                tr_hist = set(user_movies_hist.tolist())
                query_hist = list(tr_hist - set([int(r.movieId)]))
                query_hist = list2str(query_hist)
                queue.append([int(r.userId), query_hist, r.genres, r.avg_rating, r.year, int(r.movieId), r.timestamp, r.rating])
    return pd.DataFrame(queue, columns=columns)
    
tr_merged = preprocess(tr, movies)
tr_merged.to_csv('./tr.raw.movielens.csv', index=False, header=None)

te_merged = preprocess(te, movies, tr, is_train=False)
te_merged.to_csv('./te.raw.movielens.csv', index=False, header=None)
# 合併成一個檔案
merged = pd.concat([tr_merged, te_merged], ignore_index=True)
merged.to_csv('./merged_movielens.csv', index=False, header=None)
merged.head()

,user_id,query_movie_ids,genres,avg_rating,year,candidate_movie_id,timestamp,rating
0,1,"1953,2105,31,1029,1061,1129,1263,1287,1293,133...",Drama,4.260870,1989.0,1172,2009-12-14 10:53:25,1
1,1,"1172,2105,31,1029,1061,1129,1263,1287,1293,133...",Action|Crime|Thriller,4.021739,1971.0,1953,2009-12-14 10:53:11,1
2,1,"1172,1953,31,1029,1061,1129,1263,1287,1293,133...",Action|Adventure|Sci-Fi,3.478723,1982.0,2105,2009-12-14 10:52:19,1
3,1,"1172,1953,2105,1029,1061,1129,1263,1287,1293,1...",Drama,3.178571,1995.0,31,2009-12-14 10:52:24,0
4,1,"1172,1953,2105,31,1061,1129,1263,1287,1293,133...",Animation|Children|Drama|Musical,3.702381,1941.0,1029,2009-12-14 10:52:59,0


In [6]:
tr = pd.read_csv('./movielens.tr', names=loader.schema.cols)
vl = pd.read_csv('./movielens.vl', names=loader.schema.cols)

In [7]:
len(tr), len(vl)

(69717, 30287)

<br/>
<br/>
<br/>
## Read JSON Config

In [5]:
%%time
utils = reload('utils.utils')
reco = reload('reco_mf_dnn.reco_mf_dnn_flex_shema')
loader = reco.Loader(conf_path='./movielens.yaml',
                     parsed_conf_path='./parsed.yaml',
                     raw_paths=['./tr.raw.movielens.csv', './te.raw.movielens.csv'])
loader.tansform('./tr.raw.movielens.csv', './movielens.tr', chunksize=20000, reset=True)\
      .tansform('./te.raw.movielens.csv', './movielens.vl', chunksize=20000, reset=False)

try to parse ./movielens.yaml (user supplied) ...
try to transform ./tr.raw.movielens.csv ... 
[./tr.raw.movielens.csv]: process take time 0:00:38.276189
try to transform ./te.raw.movielens.csv ... 
[./te.raw.movielens.csv]: process take time 0:00:16.452941
Wall time: 57.3 s


<br/>
<br/>
<br/>
## MF + DNN

In [2]:
utils = reload('utils.utils')
reco = reload('reco_mf_dnn.reco_mf_dnn_flex_shema')

schema = reco.Schema(conf_path='./movielens.yaml',
                     parsed_conf_path='./parsed.yaml',
                     raw_paths=['./tr.raw.movielens.csv', './te.raw.movielens.csv']).init()
schema.col_states_

OrderedDict([('query_movie_ids',
              CatgMapper(allow_null=True, default=None, is_multi=True,
                    name='query_movie_ids', sep=',', vocabs=None,
                    vocabs_path='item.vocab')),
             ('genres',
              CatgMapper(allow_null=True, default=None, is_multi=True, name='genres',
                    sep='|', vocabs=None, vocabs_path='genres.vocab')),
             ('avg_rating', NumericMapper(default=None, name='avg_rating')),
             ('year', NumericMapper(default=None, name='year')),
             ('candidate_movie_id',
              CatgMapper(allow_null=True, default=None, is_multi=False,
                    name='candidate_movie_id', sep=None, vocabs=None,
                    vocabs_path='item.vocab')),
             ('rating',
              CatgMapper(allow_null=False, default=None, is_multi=False, name='rating',
                    sep=None, vocabs=None, vocabs_path=None))])

In [3]:
schema.df_conf_

,id,m_dtype,date_format,default,is_multi,sep,vocabs,vocabs_path,aux,type,col_state
id,,,,,,,,,,,
user_id,user_id,catg,None,None,False,None,None,None,False,None,None
query_movie_ids,query_movie_ids,catg,None,None,True,",",None,item.vocab,False,user,"allow_null: true\nclasses_: ['1', '2', '3', '4..."
genres,genres,catg,None,None,True,|,None,genres.vocab,False,item,allow_null: true\nclasses_: [(no genres listed...
avg_rating,avg_rating,cont,None,None,False,None,None,None,False,item,"{cumsum_: 354374.99999999965, default: null, m..."
year,year,cont,None,None,False,None,None,None,False,item,"{cumsum_: 199176755.0, default: null, max_: 20..."
candidate_movie_id,candidate_movie_id,catg,None,None,False,None,None,item.vocab,False,item,"allow_null: true\nclasses_: ['1', '2', '3', '4..."
timestamp,timestamp,datetime,%Y-%m-%d %H:%M:%S,None,False,None,None,None,False,None,None
rating,rating,catg,None,None,False,None,None,None,False,label,"allow_null: false\nclasses_: ['0', '1']\ndefau..."


In [4]:
utils = reload('utils.utils')
reco = reload('reco_mf_dnn.reco_mf_dnn_flex_shema')

model_dir='./model/reco_mf_dnn'
dim = 16
lr = 0.005
n_batch = 128
n_movies = len(schema.col_states_['candidate_movie_id'].classes_)
n_genres = len(schema.col_states_['genres'].classes_)

reco_mf_dnn = reload('reco_mf_dnn.reco_mf_dnn_flex_shema')
model = reco_mf_dnn.ModelMfDNN(schema, n_movies, n_genres, dim=dim, model_dir=model_dir)
model.fit()

./model/reco_mf_dnn
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_is_chief': True, '_model_dir': './model/reco_mf_dnn', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000021D01C18>, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_tf_random_seed': 88, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_task_id': 0, '_task_type': 'worker', '_master': '', '_save_checkpoints_steps': 545, '_session_config': None}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model/reco_mf_dnn\model.ckpt.
INFO:tensorflow:step = 1, loss = 1.5711223
INFO:tensorflow:global_step/sec: 71.9901
INFO:tensorflow

INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:step = 10051, loss = 0.09192287 (0.909 sec)
INFO:tensorflow:global_step/sec: 114.672
INFO:tensorflow:step = 10151, loss = 0.49231997 (0.871 sec)
INFO:tensorflow:global_step/sec: 115.467
INFO:tensorflow:step = 10251, loss = 0.56248546 (0.867 sec)
INFO:tensorflow:global_step/sec: 81.5614
INFO:tensorflow:step = 10351, loss = 0.69551355 (1.225 sec)
INFO:tensorflow:Saving checkpoints for 10356 into ./model/reco_mf_dnn\model.ckpt.
INFO:tensorflow:global_step/sec: 18.4764
INFO:tensorflow:step = 10451, loss = 0.09225416 (5.413 sec)
INFO:tensorflow:global_step/sec: 112.988
INFO:tensorflow:step = 10551, loss = 0.95989263 (0.885 sec)
INFO:tensorflow:global_step/sec: 80.1236
INFO:tensorflow:step = 10651, loss = 0.91996026 (1.247 sec)
INFO:tensorflow:global_step/sec: 111.228
INFO:tensorflow:step = 10751, loss = 0.42750683 (0.900 sec)
INFO:tensorflow:global_step/sec: 114.672
INFO:tensorflow:step = 10851, loss = 0.7104304 (0.871 sec)
INFO:tensorflow:Saving checkpoints for 10900 into .

KeyboardInterrupt: 

In [38]:
(30287 // 128) + 1

237

In [36]:
%%bash
cd H:/workspace/notebook/tensorflow_estimator/cloudml-samples/census/customestimator
export TRAIN_STEPS=1000
DATE=`date '+%Y%m%d_%H%M%S'`
export OUTPUT_DIR=census_model
rm -rf $OUTPUT_DIR

TRAIN_FILE=../data/adult.data.csv
EVAL_FILE=../data/adult.test.csv
python -m trainer.task --train-files $TRAIN_FILE \
                       --eval-files $EVAL_FILE \
                       --job-dir $OUTPUT_DIR \
                       --train-steps $TRAIN_STEPS \
                       --eval-steps 100

[('embedding_size', 8), ('eval_batch_size', 40), ('eval_files', ['../data/adult.test.csv']), ('eval_steps', 100), ('export_format', 'JSON'), ('first_layer_size', 100), ('job_dir', 'census_model'), ('learning_rate', 0.1), ('num_epochs', None), ('num_layers', 4), ('scale_factor', 0.7), ('train_batch_size', 40), ('train_files', ['../data/adult.data.csv']), ('train_steps', 1000), ('verbosity', 'INFO')]


H:\Anaconda3\envs\ml_dl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_master': '', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001D217DA0>, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_session_config': None, '_task_type': 'worker', '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_model_dir': 'census_model', '_save_summary_steps': 100, '_service': None, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluat

In [6]:
tr = pd.read_csv('./movielens.tr', names=schema.cols)
vl = pd.read_csv('./movielens.vl', names=schema.cols)

In [7]:
tr.head()

,query_movie_ids,genres,avg_rating,year,candidate_movie_id,rating
0,"1516,1666,31,834,860,907,1018,1042,1048,1084,1...",9,0.835749,0.763158,932,0
1,"932,1666,31,834,860,907,1018,1042,1048,1084,10...","2,7,18",0.782609,0.605263,1516,0
2,"932,1516,31,834,860,907,1018,1042,1048,1084,10...","2,3,17",0.661939,0.701754,1666,0
3,"932,1516,1666,834,860,907,1018,1042,1048,1084,...",9,0.595238,0.815789,31,1
4,"932,1516,1666,31,860,907,1018,1042,1048,1084,1...","4,5,9,14",0.711640,0.342105,834,1


In [ ]:
def make_datasets(filenames, schema, n_batch=128, n_epoch=1):
    cols = ['query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
    defaults = [[''], [''], [], [], [0], [0]]
    def parse_csv(value):
        data = tf.decode_csv(value, record_defaults=defaults)
        features = OrderedDict(zip(cols, data))
        for col in ('query_movie_ids', 'genres'):
            features[col] = tf.string_to_number(tf.string_split([features[col]], ',').values, out_type=tf.int32)
            # features[col] = utils.to_dense(features[col])
        return features

    def add_seq_cols(feat):
        multi_cols = ('query_movie_ids', 'genres')
        for m_col in multi_cols:
            name = '{}_lens'.format(m_col)
            feat['{}_lens'.format(m_col)] = tf.size(feat[m_col])
            cols.append(name)
        return feat
    
    dataset = tf.data.TextLineDataset(filenames)
    dataset = dataset.map(parse_csv, num_parallel_calls=4)
    dataset = dataset.map(add_seq_cols, num_parallel_calls=4)
    dataset = dataset.padded_batch(n_batch, OrderedDict(zip(cols, ([None], [None], [], [], [], [], [], []))) )
    dataset = dataset.shuffle(n_batch * 1000, seed=seed).repeat(n_epoch)
    features = dataset.make_one_shot_iterator().get_next()
    return features, features.pop('rating')
                                
# tf.reset_default_graph()
with tf.Graph().as_default():
    inputs = make_datasets(['./movielens.tr'], loader.schema, n_batch=5)
    query_movie_ids = inputs[0]['query_movie_ids']
    print(query_movie_ids.get_shape().as_list())
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            _, = sess.run([inputs])
            pass


In [ ]:
def make_datasets(fpath_ary, schema, n_batch=128, n_epoch=1):
    def to_dense(sp):
        dense = tf.sparse_to_dense(sp.indices, sp.dense_shape, sp.values, '')
        return tf.reshape(tf.to_int32(tf.string_to_number(dense)), [-1])

    def to_sparse(dense):
        idx = tf.where(tf.not_equal(dense, 0))
        return tf.SparseTensor(indices=idx, dense_shape=dense.get_shape(), values=tf.gather_nd(dense, idx))

    def parse_csv(value):
        data = tf.decode_csv(value, record_defaults=defaults)
        features = OrderedDict(zip(cols, data))
        multi_cols = df_conf.query("{} == '{}' and {} == True".format(schema.M_DTYPE, schema.CATG, schema.IS_MULTI)).id.values
        for col in multi_cols:
            features[col] = tf.string_split([features[col]], ',')
            features[col] = to_dense(features[col])
            # features['{}_lens'.format(col)] = tf.size(features[col])
        return features 
    
    df_conf = schema.df_conf_.query('{}.notnull()'.format(schema.TYPE))
    cols = schema.cols
    defaults = []
    for _, r in df_conf.iterrows():
        if r[schema.M_DTYPE] == schema.CATG:
            defaults.append([''] if r[schema.IS_MULTI] else [0])
        else:
            defaults.append([])
    dataset = tf.data.TextLineDataset(fpath_ary)
    dataset = dataset.map(parse_csv, num_parallel_calls=4)
    has_multi = (df_conf[schema.M_DTYPE] == schema.CATG) & (df_conf[schema.IS_MULTI] == True)
    if sum(has_multi):
        multi_cols = df_conf[has_multi].id.values
        dataset = dataset.padded_batch(n_batch, OrderedDict( zip(cols, tuple([None] if e else [] for e in has_multi))) )
    else:
        dataset = dataset.batch(n_batch)
    dataset = dataset.shuffle(n_batch * 10, seed=seed).repeat(n_epoch)
    features = dataset.make_one_shot_iterator().get_next()
    return features, features.pop(schema.label[0])
                                
# tf.reset_default_graph()
with tf.Graph().as_default():
    inputs = make_datasets(['./movielens.tr'], loader.schema, n_batch=30)
    query_lens = tf.sequence_mask([1, 2, 3])
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            _, = sess.run([inputs])
            # print( sess.run(inputs) )
            pass

## Feature Columns with tf.feature_column.input_layer

In [ ]:
a = pd.Series(minmax_scale(np.random.normal(0, 1, size=1000)))
a.hist(bins=50)

In [ ]:
%%time
tf.reset_default_graph()
with tf.Graph().as_default():
    user_id = tf.feature_column.categorical_column_with_hash_bucket('user_id', hash_bucket_size=1000, dtype=tf.int32)
    user_id = tf.feature_column.embedding_column(user_id, dimension=8)
    avg_rating = tf.feature_column.numeric_column('avg_rating')
    columns = [user_id, avg_rating]
    
    def make_datasets(fpath_ary):
        cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
        defaults = [[0], [''], [''], [], [], [0], []]

        def parse_csv(value):
            data = tf.decode_csv(value, record_defaults=defaults)
            features = OrderedDict(zip(cols, data))
            # print(features)
            return features
        
        dataset = tf.data.TextLineDataset(fpath_ary)
        dataset = (dataset.map(parse_csv, num_parallel_calls=4)
                          .batch(3)
                          # .padded_batch(3, OrderedDict(zip(cols, ([], [None], [None], [], [], [], []))))
                          .shuffle(10, seed=seed)
                          .repeat(1)
                  )
        return dataset.make_one_shot_iterator().get_next()
    
    inputs = make_datasets(['./te_processed.batch.csv'])
    inputs = tf.feature_column.input_layer(inputs, columns)
    # features = tf.parse_example(serialized_example, features=tf.feature_column.make_parse_example_spec(columns))
    ctx = []
    with tf.train.MonitoredTrainingSession() as sess:
        while not sess.should_stop():
            print(sess.run(inputs))

### Make Example

In [ ]:
%%time
cols = ['user_id', 'query_movie_ids', 'genres', 'avg_rating', 'year', 'candidate_movie_id', 'rating']
is_multi = [False, True, True, False, False, False, False]
pd_dtypes = [int, str, str, float, float, int, float]
types = ['int64_list', 'int64_list', 'int64_list', 'float_list', 'float_list', 'int64_list', 'float_list']
tf_types = [tf.int64, tf.int64, tf.int64, tf.float32, tf.float32, tf.int64, tf.float32]
def persist_example(fpath, tfpath):
    with tf.python_io.TFRecordWriter(tfpath) as w:
        for chunk in pd.read_csv(fpath, names=cols, dtype=dict(zip(cols, pd_dtypes)), chunksize=1000):
            chunk['query_movie_ids'] = chunk.query_movie_ids.map(lambda r: map(int, r.split(',')))
            chunk['genres'] = chunk.genres.map(lambda r: map(int, r.split(',')))
            
            for idx, r in chunk.iterrows():
                ex = tf.train.Example()
                for multi, col, tpe in zip(is_multi, cols, types):
                    val = r[col]
                    # ex.features.feature[col].int64_list or float_list or bytes_list
                    feat_type = getattr(ex.features.feature[col], tpe)
                    # extend function for multivalent columns, otherwise append
                    append_or_extend = 'append' if not multi else 'extend'                    
                    getattr(feat_type.value, append_or_extend)(val)
                w.write(ex.SerializePartialToString())

persist_example('./te_processed.csv', './data.tfrecord')

In [ ]:
def decode_example(ser_example):
    # queue = tf.train.string_input_producer([fpath], num_epochs=1)
    # _, ser_example = tf.TFRecordReader().read(queue)
    # ser_example = tf.train.batch([ser_example], batch_size=10)
    ctx_features = {col: tf.FixedLenFeature([], tf_tpe)
                    for col, tf_tpe in zip(cols, tf_types) if col not in ('query_movie_ids', 'genres')}
    seq_features = {col: tf.FixedLenSequenceFeature([], tf_tpe) 
                    for col, tf_tpe in [('query_movie_ids', tf.int64), ('genres', tf.int64)]}
    context_dict, sequence_dict = tf.parse_single_sequence_example(ser_example, 
                                                                   context_features=ctx_features, 
                                                                   sequence_features=seq_features)
    # for col, tpe in zip(cols, tf_types):
    #     val = feature_dict[col]
    #     feature_dict[col] = tf.sparse_to_dense(val.indices, val.dense_shape, val.values, name=col)
    feature_dict = {}
    feature_dict.update(context_dict)
    feature_dict.update(sequence_dict)
    ret = OrderedDict()
    for c in cols:
        ret[c] = feature_dict[c]
    return tuple(ret.values())

tf.reset_default_graph()
with tf.Graph().as_default():
    dataset = tf.data.TFRecordDataset(['./data.tfrecord'])
    dataset = dataset.map(decode_example).padded_batch(10, padded_shapes=([], [None], [None], [], [], [], []))
    # dataset = dataset.batch(3)
    iters = dataset.make_one_shot_iterator()
    r = iters.get_next()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        print( sess.run(r) )

## Traditional parse_example
1. tf.train.Coordinator + tf.train.start_queue_runners

In [ ]:
from tensorflow.python.framework import sparse_tensor
import re

def to_sparse(dense):
    idx = tf.where(tf.not_equal(dense, 0))
    return tf.SparseTensor(idx, tf.gather_nd(dense, idx), dense.get_shape())

def make_example(val):
    example = tf.train.Example(features=tf.train.Features(
        feature = {
            'query_movie_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=val)),
            'genres': tf.train.Feature(int64_list=tf.train.Int64List(value=val))
        }
    ))
    return example

tf.reset_default_graph()
with tf.Graph().as_default():
    
    filename = "tmp.tfrecords"
    if not os.path.exists(filename):
        # os.remove(filename)
        writer = tf.python_io.TFRecordWriter(filename)
        with writer:
            for idx, r in teProcessed.head().iterrows():
                for col in ('query_movie_ids', 'genres'):
                    val = list(map(int, re.split(',\s*', r[col])))
                    ex = make_example(val)
                    writer.write(ex.SerializeToString())

    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer(["tmp.tfrecords"], num_epochs=1)
    _, serialized_example = reader.read(filename_queue)

    batch = tf.train.batch(tensors=[serialized_example], batch_size=1)
    features = {
        'query_movie_ids': tf.VarLenFeature(tf.int64),
        'genres': tf.VarLenFeature(tf.int64)
    }
    data = tf.parse_example(batch, features)
    query_movie_ids = data['query_movie_ids']
    embbedding = tf.Variable(tf.glorot_uniform_initializer()([9125]), dtype=tf.float32)
    print(query_movie_ids.dense_shape)
    # r = tf.layers.dense(query_movie_ids, 10)
    # emb_query = tf.nn.embedding_lookup_sparse([embbedding], query_movie_ids, None, combiner='sqrtn')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        try:
            print(sess.run(data))
            pass
        except tf.errors.OutOfRangeError as e:
            coord.request_stop(e)
        finally:
            coord.request_stop()
            coord.join(threads)
    

## Test

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default():
    labels = tf.constant(np.ones([10, 8]))
    pred = tf.concat([tf.Variable(tf.ones(shape=[1, 8]), trainable=False), tf.Variable(tf.truncated_normal([9, 8]))], 0)
    loss = tf.losses.mean_squared_error(predictions=pred, labels=labels)
    train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        print(pred.eval())
        for i in range(1000):
            sess.run([train_op])
        print()
        print(pred.eval())

In [ ]:
tf.zeros